In [9]:
import pickle
import os
import numpy as np
from itertools import chain

In [ ]:


# navigate up one folder then down into the scraped folder
os.chdir('..')
os.chdir('scraped')

#use pickle to read chords_songs_0-1000.pkl
with open("chords_songs_0-999.pkl", 'rb') as f:
    upto_1000 = pickle.load(f)

#use pickle to read chords_songs_1000-1999.pkl
with open("chords_songs_1000-1999.pkl", 'rb') as f:
    upto_2000 = pickle.load(f)

#use pickle to read chords_songs_2000-2999.pkl
with open("chords_songs_2000-2999.pkl", 'rb') as f:
    upto_3000 = pickle.load(f)

#use pickle to read chords_songs_3000-4955.pkl
with open("chords_songs_3000-4955.pkl", 'rb') as f:
    upto_4955 = pickle.load(f)

# upto_1000 is a list of LISTS. Flatten it.
    
upto_3000

In [14]:
part_flattened = []
for to_flatten in [upto_1000, upto_2000, upto_3000, upto_4955]:
    part_flattened.append(list(chain(*to_flatten)))



In [16]:
# again flatten part_flattened
flattened = list(chain(*part_flattened))
len(flattened)

4

In [13]:
test1 = list(chain.from_iterable(upto_3000))
len(test1)a

88942

### Import pacakges

In [ ]:
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers